# SSM 계산
* Heatmap으로 나타내기
* 시공도로 나타내기

# Import

In [9]:
import pandas as pd
import numpy as np

from tqdm import tqdm
import warnings
import os

import math # arctangent; math.atan 사용 목적
import statistics

from SSM_time_based import *
from SSM_deceleration_based import *
from SSM_energy_based import *

# Load Dataset

In [10]:
working_dir = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/011_SSM실증분석_230315-/Dataset/'

In [11]:
folder_name = '02_processed'
folder_dir = os.path.join(working_dir, folder_name)

file_name = 'merging_type1_03frm_0.10sec.csv'
file_path = os.path.join(folder_dir, file_name)

# Calculate

In [12]:
folder_name = '03_SSM'
SSM_dir = os.path.join(working_dir, folder_name)

#os.makedirs(SSM_dir, exist_ok = True)

In [16]:
threshold_TTC = 3 # time-based SSM 임계값
threshold_DRAC = 3.4

tmsec = 0.1 # 각 row별 시간단위(초)
g = 9.81 # 중력가속도
f = 0.35 # 마찰계수. friction coeefficient
delta_t = 1 # Reaction Delay time. 운전자 반응시간
MADR = 8.45 # 차량의 최대 감속 능력. 날씨에 따라, 차종에 따라 다르다. Saccomanno et al.(2008) 참조
deceleration_rate = 0.3 * g # 차량 감속도. 3.3도 됨_MTTC)

## LV0

In [21]:
df = pd.read_csv(file_path, encoding = 'cp949')

# 거리관련 변수
df['D_x'] = df['LV0_local_x'] - df['local_x']
df['D_y'] = df['LV0_local_y'] - df['local_y']
df['D'] = (df['D_x']**2 + df['D_y']**2) ** (1/2)

df['TTC'] = df.apply(lambda x: TTC(x.local_x, x.LV0_local_x, x.velocity_x, x.LV0_velocity_x, x.V_len, x.LV0_len, x.V_wid, x.LV0_wid, x.local_y, x.LV0_local_y, x.velocity_y, x.LV0_velocity_y), axis = 1)
df['ITTC'] = df.apply(lambda x: ITTC(x.local_x, x.LV0_local_x, x.velocity_x, x.LV0_velocity_x, x.V_len, x.LV0_len, x.V_wid, x.LV0_wid, x.local_y, x.LV0_local_y, x.velocity_y, x.LV0_velocity_y), axis = 1)
df['MTTC'] = df.apply(lambda x: MTTC(x.local_x, x.LV0_local_x, x.velocity_x, x.LV0_velocity_x, x.acc_x, x.LV0_acc_x, x.V_len, x.LV0_len, x.V_wid, x.LV0_wid, x.local_y, x.LV0_local_y, x.velocity_y, x.LV0_velocity_y), axis = 1)
df['MTTC_conflict_occurs'] = df.apply(lambda x: conflict_occurs(x.local_x, x.LV0_local_x, x.velocity_x, x.LV0_velocity_x, x.acc_x, x.LV0_acc_x, x.V_wid, x.LV0_wid, x.local_y, x.LV0_local_y, x.velocity_y, x.LV0_velocity_y), axis = 1)
df['CI_MTTC'] = df.apply(lambda x: CI_MTTC(x.local_x, x.LV0_local_x, x.velocity_x, x.LV0_velocity_x, x.acc_x, x.LV0_acc_x, x.MTTC), axis = 1)

df['TIT'] = df.apply(lambda x: TIT(x.TTC, threshold_TTC, tmsec), axis = 1)
df['TET'] = df.apply(lambda x: TET(x.TTC, threshold_TTC, tmsec), axis = 1)

df['V_point_1'], df['V_point_2'], df['V_point_3'], df['V_point_4'] = zip(*df.apply(lambda x: points(x.local_x, x.local_y, x.velocity_x, x.velocity_y, x.V_wid, x.V_len), axis = 1))
df['LV0_point_1'], df['LV0_point_2'], df['LV0_point_3'], df['LV0_point_4'] = zip(*df.apply(lambda x: points(x.LV0_local_x, x.LV0_local_y, x.LV0_velocity_x, x.LV0_velocity_y, x.LV0_wid, x.LV0_len), axis = 1))

## Nearest_point 추출
df['V_LV0_point_num'], df['V_LV0_point_n'], df['LV0_point_n'], df['V_LV0_point_num2'], df['V_LV0_point_n2'], df['LV0_point_n2'] = zip(*df.apply(lambda x: nearest_point(x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, x.LV0_point_1, x.LV0_point_2, x.LV0_point_3, x.LV0_point_4), axis = 1))

## Shortest Distance 구하기 : ACT 용
df['V_LV0_point_delta'], df['V_LV0_shortest_vector'], df['V_LV0_shortest_distance'] = zip(*df.apply(lambda x: shortest_distance(
    x.V_LV0_point_num, x.V_LV0_point_n, x.LV0_point_n, 
    x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
    x.LV0_point_1, x.LV0_point_2, x.LV0_point_3, x.LV0_point_4), axis = 1))

# 2차원 SSM

df['T2'] = df.apply(lambda x: T2(
    x.V_LV0_point_num, x.V_LV0_point_n, x.LV0_point_n, 
    x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
    x.LV0_point_1, x.LV0_point_2, x.LV0_point_3, x.LV0_point_4,
    x.velocity_x, x.velocity_y, x.LV0_velocity_x, x.LV0_velocity_y), axis = 1)

df['ACT'] = df.apply(lambda x: ACT(
    x.velocity_x, x.velocity_y, x.velocity_x_before, x.velocity_y_before,
    x.LV0_velocity_x, x.LV0_velocity_y, x.LV0_velocity_x_before, x.LV0_velocity_y_before,
    x.acc_x, x.acc_y, x.LV0_acc_x, x.LV0_acc_y, 
    x.V_LV0_shortest_vector, x.V_LV0_shortest_distance), axis = 1)

df['pPET'] = df.apply(lambda x: pPET(
    x.V_LV0_point_num, x.V_LV0_point_n, x.LV0_point_n, 
    x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
    x.LV0_point_1, x.LV0_point_2, x.LV0_point_3, x.LV0_point_4,
    x.velocity_x, x.velocity_y, x.LV0_velocity_x, x.LV0_velocity_y), axis = 1)

df['TIT2'] = df.apply(lambda x: TIT(x.T2, threshold_TTC, tmsec), axis = 1)
df['ACTIT'] = df.apply(lambda x: TIT(x.ACT, threshold_TTC, tmsec), axis = 1)

df['SSCR_TTC_prime'] = df.apply(lambda x: SSCR_TTC_prime(
    x.V_LV0_point_num, x.V_LV0_point_n, x.LV0_point_n, 
    x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
    x.LV0_point_1, x.LV0_point_2, x.LV0_point_3, x.LV0_point_4,
    x.velocity_x, x.velocity_y, x.LV0_velocity_x, x.LV0_velocity_y), axis = 1)

df['SSCR_R'] = df.apply(lambda x: SSCR_R(x.TTC, x.pPET), axis = 1)
df['SSCR_Y'] = df.apply(lambda x: SSCR_Y(x.TTC, x.pPET, x.SSCR_TTC_prime), axis = 1)
df['SSCR'] = df['SSCR_R'] + df['SSCR_Y']

df = TA(df) # Time to Accident 계산
df['critical_speed'] = df.apply(lambda x: critical_speed(g, f, x.pPET), axis = 1)
df['TA_CS'] = df.apply(lambda x: TA_CS(x.TA, x.critical_speed), axis = 1) # TA/CS 계산

df['PSD'] = df.apply(lambda x: PSD(x.local_x, x.LV0_local_x, x.velocity_x, x.LV0_velocity_x, x.V_len, x.LV0_len, f, g, x.V_wid, x.LV0_wid, x.local_y, x.LV0_local_y, x.velocity_y, x.LV0_velocity_y), axis = 1)

df['DSS'] = df.apply(lambda x: DSS(x.local_x, x.LV0_local_x, x.velocity_x, x.LV0_velocity_x, x.V_len, x.LV0_len, g, f, delta_t, x.V_wid, x.LV0_wid, x.local_y, x.LV0_local_y, x.velocity_y, x.LV0_velocity_y), axis = 1)
df['TIDSS'] = df.apply(lambda x: TIDSS(x.DSS, tmsec), axis = 1)
df['PICUD'] = df.apply(lambda x: PICUD(x.local_x, x.LV0_local_x, x.velocity_x, x.LV0_velocity_x, x.D_x, delta_t, deceleration_rate, x.V_wid, x.LV0_wid, x.local_y, x.LV0_local_y, x.velocity_y, x.LV0_velocity_y), axis = 1)

df['DRAC'] = df.apply(lambda x: DRAC(x.local_x, x.LV0_local_x, x.D_x, x.velocity_x, x.LV0_velocity_x, x.LV0_len, x.V_len, x.V_wid, x.LV0_wid, x.local_y, x.LV0_local_y, x.velocity_y, x.LV0_velocity_y), axis = 1)
df['TIDRAC'] = df.apply(lambda x: TIDRAC(x.DRAC, threshold_DRAC), axis = 1)

df['MDRAC'] = df.apply(lambda x: MDRAC(x.local_x, x.LV0_local_x, x.velocity_x, x.LV0_velocity_x, x.TTC, delta_t, x.V_wid, x.LV0_wid, x.local_y, x.LV0_local_y, x.velocity_y, x.LV0_velocity_y), axis = 1)

df['MDRAC2'] = df.apply(lambda x: MDRAC(x.local_x, x.LV0_local_x, x.velocity, x.LV0_velocity, x.T2, delta_t, x.V_wid, x.LV0_wid, x.local_y, x.LV0_local_y, x.velocity_y, x.LV0_velocity_y), axis = 1)

df['DCIA'] = df.apply(lambda x: DCIA(x.local_x, x.LV0_local_x, x.velocity_x, x.LV0_velocity_x, x.acc_x, x.LV0_acc_x, delta_t, x.D_x, x.V_wid, x.LV0_wid, x.local_y, x.LV0_local_y, x.velocity_y, x.LV0_velocity_y), axis = 1)

df['unsafety'] = df.apply(lambda x: unsafety(x.local_x, x.LV0_local_x, x.velocity_x, x.LV0_velocity_x, x.acc_x, x.LV0_acc_x, deceleration_rate, x.V_wid, x.LV0_wid, x.local_y, x.LV0_local_y, x.velocity_y, x.LV0_velocity_y), axis = 1)

df['MTC'] = df.apply(lambda x: MTC(x.local_x, x.LV0_local_x, x.D_x, x.V_len, x.LV0_len, x.velocity_x, x.LV0_velocity, delta_t, deceleration_rate, x.V_wid, x.LV0_wid, x.local_y, x.LV0_local_y, x.velocity_y, x.LV0_velocity_y), axis = 1)
df['MMTC'] = df.apply(lambda x: MMTC(x.local_x, x.LV0_local_x, x.velocity_x, x.LV0_velocity_x, g, x.V_wid, x.LV0_wid, x.local_y, x.LV0_local_y, x.velocity_y, x.LV0_velocity_y), axis = 1)


df['RCRI'] = df.apply(lambda x: RCRI(x.local_x, x.LV0_local_x, x.velocity_x, x.LV0_velocity_x, x.acc_x, x.LV0_acc_x, x.D_x, delta_t, x.V_wid, x.LV0_wid, x.local_y, x.LV0_local_y, x.velocity_y, x.LV0_velocity_y), axis = 1)
df['TERCRI'] = df.apply(lambda x: TERCRI(x.RCRI, tmsec), axis = 1)

df['CIF'] = df.apply(lambda x: CIF(x.velocity_x, x.TTC), axis = 1)
df['CIF2'] = df.apply(lambda x: CIF(x.velocity, x.T2), axis = 1)
df['ACIF'] = df.apply(lambda x: CIF(x.velocity, x.ACT), axis = 1)


delete_cols = ['V_point_1', 'V_point_2', 'V_point_3', 'V_point_4', 
               'LV0_point_1', 'LV0_point_2', 'LV0_point_3', 'LV0_point_4',
               'V_LV0_point_num', 'V_LV0_point_n', 'LV0_point_n', 'V_LV0_point_num2',
               'V_LV0_point_n2',	'LV0_point_n2',	'V_LV0_point_delta',	'V_LV0_shortest_vector',
               'V_LV0_shortest_distance']

df.drop(delete_cols, inplace = True, axis = 1)

save_SSM_name = file_name[:-4] + '_LV0.csv'
save_SSM_path = os.path.join(SSM_dir, save_SSM_name)
    
df.to_csv(save_SSM_path, encoding = 'cp949')

## LVL

In [23]:
df = pd.read_csv(file_path, encoding = 'cp949')

# 거리관련 변수
df['D_x'] = df['LVL_local_x'] - df['local_x']
df['D_y'] = df['LVL_local_y'] - df['local_y']
df['D'] = (df['D_x']**2 + df['D_y']**2) ** (1/2)

df['TTC'] = df.apply(lambda x: TTC(x.local_x, x.LVL_local_x, x.velocity_x, x.LVL_velocity_x, x.V_len, x.LVL_len, x.V_wid, x.LVL_wid, x.local_y, x.LVL_local_y, x.velocity_y, x.LVL_velocity_y), axis = 1)
df['ITTC'] = df.apply(lambda x: ITTC(x.local_x, x.LVL_local_x, x.velocity_x, x.LVL_velocity_x, x.V_len, x.LVL_len, x.V_wid, x.LVL_wid, x.local_y, x.LVL_local_y, x.velocity_y, x.LVL_velocity_y), axis = 1)
df['MTTC'] = df.apply(lambda x: MTTC(x.local_x, x.LVL_local_x, x.velocity_x, x.LVL_velocity_x, x.acc_x, x.LVL_acc_x, x.V_len, x.LVL_len, x.V_wid, x.LVL_wid, x.local_y, x.LVL_local_y, x.velocity_y, x.LVL_velocity_y), axis = 1)
df['MTTC_conflict_occurs'] = df.apply(lambda x: conflict_occurs(x.local_x, x.LVL_local_x, x.velocity_x, x.LVL_velocity_x, x.acc_x, x.LVL_acc_x, x.V_wid, x.LVL_wid, x.local_y, x.LVL_local_y, x.velocity_y, x.LVL_velocity_y), axis = 1)
df['CI_MTTC'] = df.apply(lambda x: CI_MTTC(x.local_x, x.LVL_local_x, x.velocity_x, x.LVL_velocity_x, x.acc_x, x.LVL_acc_x, x.MTTC), axis = 1)

df['TIT'] = df.apply(lambda x: TIT(x.TTC, threshold_TTC, tmsec), axis = 1)
df['TET'] = df.apply(lambda x: TET(x.TTC, threshold_TTC, tmsec), axis = 1)

df['V_point_1'], df['V_point_2'], df['V_point_3'], df['V_point_4'] = zip(*df.apply(lambda x: points(x.local_x, x.local_y, x.velocity_x, x.velocity_y, x.V_wid, x.V_len), axis = 1))
df['LVL_point_1'], df['LVL_point_2'], df['LVL_point_3'], df['LVL_point_4'] = zip(*df.apply(lambda x: points(x.LVL_local_x, x.LVL_local_y, x.LVL_velocity_x, x.LVL_velocity_y, x.LVL_wid, x.LVL_len), axis = 1))

## Nearest_point 추출
df['V_LVL_point_num'], df['V_LVL_point_n'], df['LVL_point_n'], df['V_LVL_point_num2'], df['V_LVL_point_n2'], df['LVL_point_n2'] = zip(*df.apply(lambda x: nearest_point(x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, x.LVL_point_1, x.LVL_point_2, x.LVL_point_3, x.LVL_point_4), axis = 1))

## Shortest Distance 구하기 : ACT 용
df['V_LVL_point_delta'], df['V_LVL_shortest_vector'], df['V_LVL_shortest_distance'] = zip(*df.apply(lambda x: shortest_distance(
    x.V_LVL_point_num, x.V_LVL_point_n, x.LVL_point_n, 
    x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
    x.LVL_point_1, x.LVL_point_2, x.LVL_point_3, x.LVL_point_4), axis = 1))

# 2차원 SSM

df['T2'] = df.apply(lambda x: T2(
    x.V_LVL_point_num, x.V_LVL_point_n, x.LVL_point_n, 
    x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
    x.LVL_point_1, x.LVL_point_2, x.LVL_point_3, x.LVL_point_4,
    x.velocity_x, x.velocity_y, x.LVL_velocity_x, x.LVL_velocity_y), axis = 1)

df['ACT'] = df.apply(lambda x: ACT(
    x.velocity_x, x.velocity_y, x.velocity_x_before, x.velocity_y_before,
    x.LVL_velocity_x, x.LVL_velocity_y, x.LVL_velocity_x_before, x.LVL_velocity_y_before,
    x.acc_x, x.acc_y, x.LVL_acc_x, x.LVL_acc_y, 
    x.V_LVL_shortest_vector, x.V_LVL_shortest_distance), axis = 1)

df['pPET'] = df.apply(lambda x: pPET(
    x.V_LVL_point_num, x.V_LVL_point_n, x.LVL_point_n, 
    x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
    x.LVL_point_1, x.LVL_point_2, x.LVL_point_3, x.LVL_point_4,
    x.velocity_x, x.velocity_y, x.LVL_velocity_x, x.LVL_velocity_y), axis = 1)

df['TIT2'] = df.apply(lambda x: TIT(x.T2, threshold_TTC, tmsec), axis = 1)
df['ACTIT'] = df.apply(lambda x: TIT(x.ACT, threshold_TTC, tmsec), axis = 1)

df['SSCR_TTC_prime'] = df.apply(lambda x: SSCR_TTC_prime(
    x.V_LVL_point_num, x.V_LVL_point_n, x.LVL_point_n, 
    x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
    x.LVL_point_1, x.LVL_point_2, x.LVL_point_3, x.LVL_point_4,
    x.velocity_x, x.velocity_y, x.LVL_velocity_x, x.LVL_velocity_y), axis = 1)

df['SSCR_R'] = df.apply(lambda x: SSCR_R(x.TTC, x.pPET), axis = 1)
df['SSCR_Y'] = df.apply(lambda x: SSCR_Y(x.TTC, x.pPET, x.SSCR_TTC_prime), axis = 1)
df['SSCR'] = df['SSCR_R'] + df['SSCR_Y']

df = TA(df) # Time to Accident 계산
df['critical_speed'] = df.apply(lambda x: critical_speed(g, f, x.pPET), axis = 1)
df['TA_CS'] = df.apply(lambda x: TA_CS(x.TA, x.critical_speed), axis = 1) # TA/CS 계산

df['PSD'] = df.apply(lambda x: PSD(x.local_x, x.LVL_local_x, x.velocity_x, x.LVL_velocity_x, x.V_len, x.LVL_len, f, g, x.V_wid, x.LVL_wid, x.local_y, x.LVL_local_y, x.velocity_y, x.LVL_velocity_y), axis = 1)

df['DSS'] = df.apply(lambda x: DSS(x.local_x, x.LVL_local_x, x.velocity_x, x.LVL_velocity_x, x.V_len, x.LVL_len, g, f, delta_t, x.V_wid, x.LVL_wid, x.local_y, x.LVL_local_y, x.velocity_y, x.LVL_velocity_y), axis = 1)
df['TIDSS'] = df.apply(lambda x: TIDSS(x.DSS, tmsec), axis = 1)
df['PICUD'] = df.apply(lambda x: PICUD(x.local_x, x.LVL_local_x, x.velocity_x, x.LVL_velocity_x, x.D_x, delta_t, deceleration_rate, x.V_wid, x.LVL_wid, x.local_y, x.LVL_local_y, x.velocity_y, x.LVL_velocity_y), axis = 1)

df['DRAC'] = df.apply(lambda x: DRAC(x.local_x, x.LVL_local_x, x.D_x, x.velocity_x, x.LVL_velocity_x, x.LVL_len, x.V_len, x.V_wid, x.LVL_wid, x.local_y, x.LVL_local_y, x.velocity_y, x.LVL_velocity_y), axis = 1)
df['TIDRAC'] = df.apply(lambda x: TIDRAC(x.DRAC, threshold_DRAC), axis = 1)

df['MDRAC'] = df.apply(lambda x: MDRAC(x.local_x, x.LVL_local_x, x.velocity_x, x.LVL_velocity_x, x.TTC, delta_t, x.V_wid, x.LVL_wid, x.local_y, x.LVL_local_y, x.velocity_y, x.LVL_velocity_y), axis = 1)

df['MDRAC2'] = df.apply(lambda x: MDRAC(x.local_x, x.LVL_local_x, x.velocity, x.LVL_velocity, x.T2, delta_t, x.V_wid, x.LVL_wid, x.local_y, x.LVL_local_y, x.velocity_y, x.LVL_velocity_y), axis = 1)

df['DCIA'] = df.apply(lambda x: DCIA(x.local_x, x.LVL_local_x, x.velocity_x, x.LVL_velocity_x, x.acc_x, x.LVL_acc_x, delta_t, x.D_x, x.V_wid, x.LVL_wid, x.local_y, x.LVL_local_y, x.velocity_y, x.LVL_velocity_y), axis = 1)

df['unsafety'] = df.apply(lambda x: unsafety(x.local_x, x.LVL_local_x, x.velocity_x, x.LVL_velocity_x, x.acc_x, x.LVL_acc_x, deceleration_rate, x.V_wid, x.LVL_wid, x.local_y, x.LVL_local_y, x.velocity_y, x.LVL_velocity_y), axis = 1)

df['MTC'] = df.apply(lambda x: MTC(x.local_x, x.LVL_local_x, x.D_x, x.V_len, x.LVL_len, x.velocity_x, x.LVL_velocity, delta_t, deceleration_rate, x.V_wid, x.LVL_wid, x.local_y, x.LVL_local_y, x.velocity_y, x.LVL_velocity_y), axis = 1)
df['MMTC'] = df.apply(lambda x: MMTC(x.local_x, x.LVL_local_x, x.velocity_x, x.LVL_velocity_x, g, x.V_wid, x.LVL_wid, x.local_y, x.LVL_local_y, x.velocity_y, x.LVL_velocity_y), axis = 1)


df['RCRI'] = df.apply(lambda x: RCRI(x.local_x, x.LVL_local_x, x.velocity_x, x.LVL_velocity_x, x.acc_x, x.LVL_acc_x, x.D_x, delta_t, x.V_wid, x.LVL_wid, x.local_y, x.LVL_local_y, x.velocity_y, x.LVL_velocity_y), axis = 1)
df['TERCRI'] = df.apply(lambda x: TERCRI(x.RCRI, tmsec), axis = 1)

df['CIF'] = df.apply(lambda x: CIF(x.velocity_x, x.TTC), axis = 1)
df['CIF2'] = df.apply(lambda x: CIF(x.velocity, x.T2), axis = 1)
df['ACIF'] = df.apply(lambda x: CIF(x.velocity, x.ACT), axis = 1)



delete_cols = ['V_point_1', 'V_point_2', 'V_point_3', 'V_point_4', 
               'LVL_point_1', 'LVL_point_2', 'LVL_point_3', 'LVL_point_4',
               'V_LVL_point_num', 'V_LVL_point_n', 'LVL_point_n', 'V_LVL_point_num2',
               'V_LVL_point_n2',	'LVL_point_n2',	'V_LVL_point_delta',	'V_LVL_shortest_vector',
               'V_LVL_shortest_distance']

df.drop(delete_cols, inplace = True, axis = 1)

save_SSM_name = file_name[:-4] + '_LVL.csv'
save_SSM_path = os.path.join(SSM_dir, save_SSM_name)
    
df.to_csv(save_SSM_path, encoding = 'cp949')

## LVR

In [24]:
df = pd.read_csv(file_path, encoding = 'cp949')

# 거리관련 변수
df['D_x'] = df['LVR_local_x'] - df['local_x']
df['D_y'] = df['LVR_local_y'] - df['local_y']
df['D'] = (df['D_x']**2 + df['D_y']**2) ** (1/2)

df['TTC'] = df.apply(lambda x: TTC(x.local_x, x.LVR_local_x, x.velocity_x, x.LVR_velocity_x, x.V_len, x.LVR_len, x.V_wid, x.LVR_wid, x.local_y, x.LVR_local_y, x.velocity_y, x.LVR_velocity_y), axis = 1)
df['ITTC'] = df.apply(lambda x: ITTC(x.local_x, x.LVR_local_x, x.velocity_x, x.LVR_velocity_x, x.V_len, x.LVR_len, x.V_wid, x.LVR_wid, x.local_y, x.LVR_local_y, x.velocity_y, x.LVR_velocity_y), axis = 1)
df['MTTC'] = df.apply(lambda x: MTTC(x.local_x, x.LVR_local_x, x.velocity_x, x.LVR_velocity_x, x.acc_x, x.LVR_acc_x, x.V_len, x.LVR_len, x.V_wid, x.LVR_wid, x.local_y, x.LVR_local_y, x.velocity_y, x.LVR_velocity_y), axis = 1)
df['MTTC_conflict_occurs'] = df.apply(lambda x: conflict_occurs(x.local_x, x.LVR_local_x, x.velocity_x, x.LVR_velocity_x, x.acc_x, x.LVR_acc_x, x.V_wid, x.LVR_wid, x.local_y, x.LVR_local_y, x.velocity_y, x.LVR_velocity_y), axis = 1)
df['CI_MTTC'] = df.apply(lambda x: CI_MTTC(x.local_x, x.LVR_local_x, x.velocity_x, x.LVR_velocity_x, x.acc_x, x.LVR_acc_x, x.MTTC), axis = 1)

df['TIT'] = df.apply(lambda x: TIT(x.TTC, threshold_TTC, tmsec), axis = 1)
df['TET'] = df.apply(lambda x: TET(x.TTC, threshold_TTC, tmsec), axis = 1)

df['V_point_1'], df['V_point_2'], df['V_point_3'], df['V_point_4'] = zip(*df.apply(lambda x: points(x.local_x, x.local_y, x.velocity_x, x.velocity_y, x.V_wid, x.V_len), axis = 1))
df['LVR_point_1'], df['LVR_point_2'], df['LVR_point_3'], df['LVR_point_4'] = zip(*df.apply(lambda x: points(x.LVR_local_x, x.LVR_local_y, x.LVR_velocity_x, x.LVR_velocity_y, x.LVR_wid, x.LVR_len), axis = 1))

## Nearest_point 추출
df['V_LVR_point_num'], df['V_LVR_point_n'], df['LVR_point_n'], df['V_LVR_point_num2'], df['V_LVR_point_n2'], df['LVR_point_n2'] = zip(*df.apply(lambda x: nearest_point(x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, x.LVR_point_1, x.LVR_point_2, x.LVR_point_3, x.LVR_point_4), axis = 1))

## Shortest Distance 구하기 : ACT 용
df['V_LVR_point_delta'], df['V_LVR_shortest_vector'], df['V_LVR_shortest_distance'] = zip(*df.apply(lambda x: shortest_distance(
    x.V_LVR_point_num, x.V_LVR_point_n, x.LVR_point_n, 
    x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
    x.LVR_point_1, x.LVR_point_2, x.LVR_point_3, x.LVR_point_4), axis = 1))

# 2차원 SSM

df['T2'] = df.apply(lambda x: T2(
    x.V_LVR_point_num, x.V_LVR_point_n, x.LVR_point_n, 
    x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
    x.LVR_point_1, x.LVR_point_2, x.LVR_point_3, x.LVR_point_4,
    x.velocity_x, x.velocity_y, x.LVR_velocity_x, x.LVR_velocity_y), axis = 1)

df['ACT'] = df.apply(lambda x: ACT(
    x.velocity_x, x.velocity_y, x.velocity_x_before, x.velocity_y_before,
    x.LVR_velocity_x, x.LVR_velocity_y, x.LVR_velocity_x_before, x.LVR_velocity_y_before,
    x.acc_x, x.acc_y, x.LVR_acc_x, x.LVR_acc_y, 
    x.V_LVR_shortest_vector, x.V_LVR_shortest_distance), axis = 1)

df['pPET'] = df.apply(lambda x: pPET(
    x.V_LVR_point_num, x.V_LVR_point_n, x.LVR_point_n, 
    x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
    x.LVR_point_1, x.LVR_point_2, x.LVR_point_3, x.LVR_point_4,
    x.velocity_x, x.velocity_y, x.LVR_velocity_x, x.LVR_velocity_y), axis = 1)

df['TIT2'] = df.apply(lambda x: TIT(x.T2, threshold_TTC, tmsec), axis = 1)
df['ACTIT'] = df.apply(lambda x: TIT(x.ACT, threshold_TTC, tmsec), axis = 1)

df['SSCR_TTC_prime'] = df.apply(lambda x: SSCR_TTC_prime(
    x.V_LVR_point_num, x.V_LVR_point_n, x.LVR_point_n, 
    x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
    x.LVR_point_1, x.LVR_point_2, x.LVR_point_3, x.LVR_point_4,
    x.velocity_x, x.velocity_y, x.LVR_velocity_x, x.LVR_velocity_y), axis = 1)

df['SSCR_R'] = df.apply(lambda x: SSCR_R(x.TTC, x.pPET), axis = 1)
df['SSCR_Y'] = df.apply(lambda x: SSCR_Y(x.TTC, x.pPET, x.SSCR_TTC_prime), axis = 1)
df['SSCR'] = df['SSCR_R'] + df['SSCR_Y']

df = TA(df) # Time to Accident 계산
df['critical_speed'] = df.apply(lambda x: critical_speed(g, f, x.pPET), axis = 1)
df['TA_CS'] = df.apply(lambda x: TA_CS(x.TA, x.critical_speed), axis = 1) # TA/CS 계산

df['PSD'] = df.apply(lambda x: PSD(x.local_x, x.LVR_local_x, x.velocity_x, x.LVR_velocity_x, x.V_len, x.LVR_len, f, g, x.V_wid, x.LVR_wid, x.local_y, x.LVR_local_y, x.velocity_y, x.LVR_velocity_y), axis = 1)

df['DSS'] = df.apply(lambda x: DSS(x.local_x, x.LVR_local_x, x.velocity_x, x.LVR_velocity_x, x.V_len, x.LVR_len, g, f, delta_t, x.V_wid, x.LVR_wid, x.local_y, x.LVR_local_y, x.velocity_y, x.LVR_velocity_y), axis = 1)
df['TIDSS'] = df.apply(lambda x: TIDSS(x.DSS, tmsec), axis = 1)
df['PICUD'] = df.apply(lambda x: PICUD(x.local_x, x.LVR_local_x, x.velocity_x, x.LVR_velocity_x, x.D_x, delta_t, deceleration_rate, x.V_wid, x.LVR_wid, x.local_y, x.LVR_local_y, x.velocity_y, x.LVR_velocity_y), axis = 1)

df['DRAC'] = df.apply(lambda x: DRAC(x.local_x, x.LVR_local_x, x.D_x, x.velocity_x, x.LVR_velocity_x, x.LVR_len, x.V_len, x.V_wid, x.LVR_wid, x.local_y, x.LVR_local_y, x.velocity_y, x.LVR_velocity_y), axis = 1)
df['TIDRAC'] = df.apply(lambda x: TIDRAC(x.DRAC, threshold_DRAC), axis = 1)

df['MDRAC'] = df.apply(lambda x: MDRAC(x.local_x, x.LVR_local_x, x.velocity_x, x.LVR_velocity_x, x.TTC, delta_t, x.V_wid, x.LVR_wid, x.local_y, x.LVR_local_y, x.velocity_y, x.LVR_velocity_y), axis = 1)

df['MDRAC2'] = df.apply(lambda x: MDRAC(x.local_x, x.LVR_local_x, x.velocity, x.LVR_velocity, x.T2, delta_t, x.V_wid, x.LVR_wid, x.local_y, x.LVR_local_y, x.velocity_y, x.LVR_velocity_y), axis = 1)

df['DCIA'] = df.apply(lambda x: DCIA(x.local_x, x.LVR_local_x, x.velocity_x, x.LVR_velocity_x, x.acc_x, x.LVR_acc_x, delta_t, x.D_x, x.V_wid, x.LVR_wid, x.local_y, x.LVR_local_y, x.velocity_y, x.LVR_velocity_y), axis = 1)

df['unsafety'] = df.apply(lambda x: unsafety(x.local_x, x.LVR_local_x, x.velocity_x, x.LVR_velocity_x, x.acc_x, x.LVR_acc_x, deceleration_rate, x.V_wid, x.LVR_wid, x.local_y, x.LVR_local_y, x.velocity_y, x.LVR_velocity_y), axis = 1)

df['MTC'] = df.apply(lambda x: MTC(x.local_x, x.LVR_local_x, x.D_x, x.V_len, x.LVR_len, x.velocity_x, x.LVR_velocity, delta_t, deceleration_rate, x.V_wid, x.LVR_wid, x.local_y, x.LVR_local_y, x.velocity_y, x.LVR_velocity_y), axis = 1)
df['MMTC'] = df.apply(lambda x: MMTC(x.local_x, x.LVR_local_x, x.velocity_x, x.LVR_velocity_x, g, x.V_wid, x.LVR_wid, x.local_y, x.LVR_local_y, x.velocity_y, x.LVR_velocity_y), axis = 1)


df['RCRI'] = df.apply(lambda x: RCRI(x.local_x, x.LVR_local_x, x.velocity_x, x.LVR_velocity_x, x.acc_x, x.LVR_acc_x, x.D_x, delta_t, x.V_wid, x.LVR_wid, x.local_y, x.LVR_local_y, x.velocity_y, x.LVR_velocity_y), axis = 1)
df['TERCRI'] = df.apply(lambda x: TERCRI(x.RCRI, tmsec), axis = 1)

df['CIF'] = df.apply(lambda x: CIF(x.velocity_x, x.TTC), axis = 1)
df['CIF2'] = df.apply(lambda x: CIF(x.velocity, x.T2), axis = 1)
df['ACIF'] = df.apply(lambda x: CIF(x.velocity, x.ACT), axis = 1)


delete_cols = ['V_point_1', 'V_point_2', 'V_point_3', 'V_point_4', 
               'LVR_point_1', 'LVR_point_2', 'LVR_point_3', 'LVR_point_4',
               'V_LVR_point_num', 'V_LVR_point_n', 'LVR_point_n', 'V_LVR_point_num2',
               'V_LVR_point_n2',	'LVR_point_n2',	'V_LVR_point_delta',	'V_LVR_shortest_vector',
               'V_LVR_shortest_distance']

df.drop(delete_cols, inplace = True, axis = 1)

save_SSM_name = file_name[:-4] + '_LVR.csv'
save_SSM_path = os.path.join(SSM_dir, save_SSM_name)
    
df.to_csv(save_SSM_path, encoding = 'cp949')